# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

from CMEP import CMEP_Atom
from qsps import QSP_HNC
from constants import *
# from hnc_Ng import  HNC_solver
# from qsps import *

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
        
        
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

Ha_to_J = 4.359744e-18
k_B = 1.380649e-23 
m_p = 1.672621898e-27 # kg
eV_to_K = 11604.5250061598
J_to_erg = 1e7
Ha_to_erg = Ha_to_J*J_to_erg

# Function that sets up HNC QSP 

## Load TCCW Cases

In [2]:
element='He'

if element=='Xe':
    ni_cc = 1.25e20
    Te_eV = 16.64e3/eV_to_K
    Ti_eV = 10.47e3/eV_to_K
    Z, A = 54, 131

elif element=='He':
    ni_cc = 1.83e21 # 1/m^3
    Te_eV = 14.48e3/eV_to_K
    Ti_eV = 11.66e3/eV_to_K
    Z, A = 2,4

ni_AU = ni_cc*1e6*aB**3

r_s_AU = QSP_HNC.rs_from_n(ni_AU)
r_c = 0
if r_c>1:
    r_c=1e-5
print("Te = {0:.3e} eV, n_i = {1:.3e} 1/cc".format(Te_eV, ni_cc))
print("Te = {0:.3e} K, ρ = {1:.3e} [g/cc] ".format(Te_eV*eV_to_K, 1e3*ni_cc*m_p*A ))
print("r_s = {0:.3f}, r_c/r_s = {1:.3f}".format(r_s_AU, r_c))

Te = 1.248e+00 eV, n_i = 1.830e+21 1/cc
Te = 1.448e+04 K, ρ = 1.224e-02 [g/cc] 
r_s = 9.584, r_c/r_s = 0.000


In [3]:
np.geomspace(1,20,num=20)[::-1]

array([20.        , 17.08262993, 14.59081227, 12.46247232, 10.64459014,
        9.09187971,  7.76566082,  6.6328955 ,  5.66536496,  4.83896665,
        4.13311383,  3.5302227 ,  3.0152744 ,  2.57544084,  2.19976514,
        1.87888869,  1.60481801,  1.37072561,  1.17077991,  1.        ])

In [9]:
T_factors = np.geomspace(1,1,num=1)[::-1]
c_list = []
T_matrix_list = []
atom_list = []
for i, T_factor in enumerate(T_factors):
    atom = CMEP_Atom(Z, A, ni_cc, T_factor*Ti_eV, T_factor*Te_eV, Zbar=Z,
                     βu_options = {'add_bridge':True, 'pseudopotential':True}, 
                     qsp_options= {'r_c':0.3},
                     hnc_options= {'kappa_multiscale':100, 'R_max':5,'N_bins':500, 'h_max':1e4},
                     hnc_solve_options={'alpha_Picard':0.1,'tol':1e-8})
    if i<2:
        atom.run_hnc()
    else:
        slope = (c_list[-1]-c_list[-2])/(T_matrix_list[-1][:,:,np.newaxis]-T_matrix_list[-2][:,:,np.newaxis])
        c_s_k_guess = c_list[-1] + slope*(atom.hnc.Temp_matrix[:,:,np.newaxis]-T_matrix_list[-1][:,:,np.newaxis])
#         atom.run_hnc(c_s_k_guess = 1/atom.hnc.Temp_matrix[:,:,np.newaxis]*Tc_s_k_guess)
        atom.run_hnc(c_s_k_guess = c_s_k_guess)
        
#     Tc_list.append(atom.hnc.Temp_matrix[:,:,np.newaxis]*atom.hnc.c_s_k_matrix)
    c_list.append(atom.hnc.c_s_k_matrix.copy())
    T_matrix_list.append(atom.hnc.Temp_matrix.copy())
    atom_list.append(atom)
    
    

Te_in_eV: 1.248
Ti_in_eV: 1.005
0: Initial condition Total Error: 2.039e+01
QUIT: Last 10 iterations error has been increasing
Exiting status 2, reverting to best index so far: 69
Final iter:79, Total Error: 6.179e-02

Root Finder:  True The solution converged. final err: 1.567e-10


In [10]:
atom.run_hnc()
atom.get_cv(ε_derivative=1e-7, ε_table=1e-6, Zbar_fixed=True)

0: Initial condition Total Error: 2.039e+01
QUIT: Last 10 iterations error has been increasing
Exiting status 2, reverting to best index so far: 69
Final iter:79, Total Error: 6.179e-02

Root Finder:  True The solution converged. final err: 1.567e-10
Te: 1.248 eV, Zbar = 2.000
0: Initial condition Total Error: 4.405e-07
QUIT: Last 10 iterations error has been increasing
Exiting status 2, reverting to best index so far: 11
Final iter:21, Total Error: 1.825e-07

Root Finder:  True The solution converged. final err: 8.341e-12
Te: 1.248 eV, Zbar = 2.000
0: Initial condition Total Error: 1.970e-06
QUIT: Last 10 iterations error has been increasing
Exiting status 2, reverting to best index so far: 17
Final iter:27, Total Error: 1.035e-06

Root Finder:  True The solution converged. final err: 4.239e-12
Te: 1.248 eV, Zbar = 2.000
0: Initial condition Total Error: 1.970e-06
QUIT: Last 10 iterations error has been increasing
Exiting status 2, reverting to best index so far: 17
Final iter:27, Tot

Seeing if full Zbar is sensitive to differing r_c

In [ ]:
# r_c = 0.1
Heat Capacity Results 
c^e_v = 3.261e+03 [J/m^3/K], c^i_v  = 3.512e+04 [J/m^3/K] 
c_v = 3.838e+04 [J/m^3/K] 

c^e_v/(ne kB)= 0.279 , c^i_v/(ni kB)   = 1.390 
c_v/(n kB) = 1.039  

In [ ]:
# r_c = 0
Heat Capacity Results 
c^e_v = 1.864e+03 [J/m^3/K], c^i_v  = 3.508e+04 [J/m^3/K] 
c_v = 3.695e+04 [J/m^3/K] 

c^e_v/(ne kB)= 0.160 , c^i_v/(ni kB)   = 1.389 
c_v/(n kB) = 1.000 

In [6]:
atom.get_effective_ion_cp()

0: Initial condition Total Error: 2.440e-03
Exiting status 0, reverting to best index so far: 6
Final iter:8, Total Error: 5.909e-09
0: Initial condition Total Error: 4.382e-03
Exiting status 0, reverting to best index so far: 8
Final iter:10, Total Error: 6.985e-09
0: Initial condition Total Error: 4.456e-03
Exiting status 0, reverting to best index so far: 8
Final iter:10, Total Error: 8.409e-09
0: Initial condition Total Error: 2.368e-03
Exiting status 0, reverting to best index so far: 6
Final iter:8, Total Error: 5.476e-09

_____________________________
Heat Capacity Results (Effective Ion Picture) 
c_p = 3.103e+04 [J/m^3/K] = 3.103e+05 [erg/cc/K]
c_p^ideal = 2.071e+04 [J/m^3/K] = 2.071e+05 [erg/cc/K]
c_v = 1.801e+04 [J/m^3/K] = 1.801e+05 [erg/cc/K]

γ = cp/cv = 1.723e+00

E/nkBT = 2.027, P/nkBT = 1.637 
cp/nkB = 3.746, cv/nkB = 2.174 
Total cv/nkB estimate (add ideal electrons):
c_v_tot_estimate = 1.837
